<a href="https://colab.research.google.com/github/YolaYing/zk-toolkit/blob/main/Plonk%2BKZG_Implementation(Python_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Develope Environment Preparation
Curve we used is BLS12-381
- Library we used: BLS21-381 curve implemented by Ethereum, using Python
- Lib link: https://github.com/ethereum/py_ecc/tree/main

You can use the following command to install all the packages we needed. Note that if you are using anaconda, package installation may be failed. Highly recommand using colab or some development friendly environment.

In [1]:
!pip3 install py_ecc

# Preliminaries


## Form of Polynomial Representations(using baby parameters for demonstration)

A polynomial $\Phi(x) = \sum_{i = 0}^{n - 1} \phi_ix^i$ have two representation forms:
1. Coefficient Form
  - $\Phi(x)$ can be represeneted as a tuple of $n$ coefficients: $[\phi_0, \phi_1, ..., \phi_{n-1}]$
2. Evaluation Form
  - $\Phi(x)$ can be represeneted as a tuple of $n$ distinct evaluations: $[\Phi(x_0), \Phi(x_1), ..., \Phi(x_{n-1})]$
    - the set of values $\\{ x_0, x_1, ..., x_{n-1} \\}$ over which the polynomial is defined over is known as "evaluation domain"

In [2]:
# For example, assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 and some points on the polynomials
q = 17
poly_coefs = [2, 3, 5, 4]
evaluation_domain = [1, 4, 16, 13]
poly_evals = [14, 10, 0, 1]

print(f'coefficient form of phi(x) is {poly_coefs}')
print(f'evaluation form of phi(x) is {poly_evals}, defined over evaluation domain {evaluation_domain}')

coefficient form of phi(x) is [2, 3, 5, 4]
evaluation form of phi(x) is [14, 10, 0, 1], defined over evaluation domain [1, 4, 16, 13]


## Convert between Coefficient Form and Evaluation Form

### Defination
- **Fourier Transform**: convert from coefficient to evaluation
- **inverse Fourier Transform**: convert from evaluation to coefficient


### Naive Way
In naive way, each of those two transformation takes $O(n^2)$ computation
- **Fourier Transform**: evaluate $\Phi(x)$ at each $x_i$ in the evaluation domain
- **inverse Fourier Transform**: use *Lagrange Interpolation* to obtain unique degree $(n-1)$ polynomial passes through each of the $n$ points


### Optimized Way: Fast Fourier Transformation(FFT)
To improve the effectiveness of the transformation, we need to do the following steps:
- defined polynomials over finite field
  - restrict each coefficient $p_i \in F_q$ and each evaluation point $\Phi(x_i) \in F_q$
  - $q$ stands for curve order
- defined the evaluation domain as a multiplicative subgroup of $F_q$
  - evaluation domain is a set of "$n^{th}$ roots of unity", $\{ \omega_0, \omega_1, ..., \omega_{n-1} \}$ for some element $\omega \in F_q$ with order $n$(i.e. $\omega^n = 1$ mod $q$)
- implementation of FFT algorithm
  - if you are not familiar with FFT, highly recommend this video: https://www.youtube.com/watch?v=h7apO7q16V0

In [3]:
import random

def FFT(poly_coefs, q):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_coefs)
  if n == 1:
    return poly_coefs

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = find_nth_root_of_unity(n, q)
  print(f'{n}^th of unity is {omega}')
  poly_coefs_e = poly_coefs[::2]
  poly_coefs_o = poly_coefs[1::2]
  y_e = FFT(poly_coefs_e, q)
  y_o = FFT(poly_coefs_o, q)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega**j)*y_o[j]) % q
    y[j + int(n/2)] = int(y_e[j] - (omega**j)*y_o[j]) % q
  return y


def find_nth_root_of_unity(n, q):
  '''
  Args:
  n: nth root of unity, which is the degree of polynomial
  q: finite field q

  Returns:
  omega: the nth root of unity, which is a element in finite field
  '''
  omega = 1
  while(omega**(n/2) == 1 and omega != 0):
    x = random.randint(0, q)
    omega = x**((q-1)/n) % q
  return omega

print(f'coefficient form of phi(x) is {poly_coefs}, after FFT, we can get the evaluation form of phi(x) is {FFT(poly_coefs, 17)}')


4^th of unity is 0.0
2^th of unity is 16.0
2^th of unity is 16.0
coefficient form of phi(x) is [2, 3, 5, 4], after FFT, we can get the evaluation form of phi(x) is [14, 14, 0, 14]


In [4]:
# verification of FFT result
omega = 13
q = 17
for i in range(len(poly_coefs)):
    x = omega**i % q
    print(f'x = {x}, poly(x) = {(4*(x**3)+5*(x**2)+3*x+2) % q}')

x = 1, poly(x) = 14
x = 13, poly(x) = 1
x = 16, poly(x) = 0
x = 4, poly(x) = 10


Because the calculation of finding evaluation domain is a repeated and computation intensive step, so usually we just build a lookup table to store the pre-calculated result. Here we just slightly revise FFT algorithm to meet the lookup needs.

In [5]:
# omega generation can be quite computational intensive, so we tend to pre-calculate the lookup table for omega
# we assume n = 8, and we have a multiplication cyclic group with ord = 8, which is [1,2,4,8,16,15,13,9] with 2 as its generator and 17 as module

# create a lookup table
n_max = 8
q = 17
omega_list = [1,2,4,8,16,15,13,9]

def build_lookup_table(n, omega_list):
  lookup = {}
  lookup[n] = omega_list
  while n > 2:
    n = int(n/2)
    omega_list = omega_list[::2]
    lookup[n] = omega_list
  return lookup

lookup  = build_lookup_table(n_max, omega_list)
print(f'lookup table of list of omega is {lookup}')

lookup table of list of omega is {8: [1, 2, 4, 8, 16, 15, 13, 9], 4: [1, 4, 16, 13], 2: [1, 16]}


In [6]:
# revise FFT using omega lookup table
def FFT_using_exist_omega(poly_coefs, q, lookup):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial
  lookup: pre-determined omega list

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_coefs)
  if n == 1:
    return poly_coefs

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = lookup[n][1]
  # print(f'n = {n}, omega = {omega}')
  poly_coefs_e = poly_coefs[::2]
  poly_coefs_o = poly_coefs[1::2]
  y_e = FFT_using_exist_omega(poly_coefs_e, q, lookup)
  y_o = FFT_using_exist_omega(poly_coefs_o, q, lookup)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega**j)*y_o[j]) % q
    y[j + int(n/2)] = int(y_e[j] - (omega**j)*y_o[j]) % q
  return y

print(f'coefficient form of phi(x) is {poly_coefs}, after FFT(use exist omega lookup table), we can get the evaluation form of phi(x) is {FFT_using_exist_omega(poly_coefs, q, lookup)}')

coefficient form of phi(x) is [2, 3, 5, 4], after FFT(use exist omega lookup table), we can get the evaluation form of phi(x) is [14, 10, 0, 1]


### Inverse Fast Fourier Transformation(IFFT)
We have used FFT to achieve fast transformation from coefficient to evaluation. Now we will implement the inverse transformation, which is from evaluation to coefficient with slightly changing in the algorithm: update $\omega$ to $\frac{1}{n}\omega^{-1}$

detailed info: https://decentralizedthoughts.github.io/2023-09-01-FFT/#mjx-eqn-%5Cstar

In [7]:
# inverse the lookup table
# the only thing we need to modify in FFT is to update its omega
omega_list = [1,2,4,8,16,15,13,9]

def inverse_element(omega_list, q):
  inverse_omega_list = []
  inverse_omega_dict = {}

  for i in omega_list:
    for candidate in omega_list:
      if i*candidate%q == 1:
        inverse_omega_list.append(candidate)
        inverse_omega_dict[i] = candidate
  return inverse_omega_list, inverse_omega_dict

inverse_omega_list, inverse_omega_dict= inverse_element(omega_list, q)
print(f'inverse omega list is {inverse_omega_list}, inverse omega dict is {inverse_omega_dict}')
inverse_lookup = build_lookup_table(n_max, inverse_omega_list)
print(f'lookup table of list of omega is {inverse_lookup}')

inverse omega list is [1, 9, 13, 15, 16, 8, 4, 2], inverse omega dict is {1: 1, 2: 9, 4: 13, 8: 15, 16: 16, 15: 8, 13: 4, 9: 2}
lookup table of list of omega is {8: [1, 9, 13, 15, 16, 8, 4, 2], 4: [1, 13, 16, 4], 2: [1, 16]}


In [8]:
# revise FFT using omega lookup table
def IFFT_using_exist_omega(poly_evals, q, inverse_lookup, inverse_omega_dict):
  recursion_result = IFFT_recursion_part(poly_evals, q, inverse_lookup)
  IFFT_final_result = [ x * inverse_omega_dict[len(poly_evals)] % q for x in recursion_result]
  return IFFT_final_result

def IFFT_recursion_part(poly_evals, q, inverse_lookup):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial
  lookup: pre-determined omega list

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_evals)
  if n == 1:
    return poly_evals

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = inverse_lookup[n][1]
  poly_evals_e = poly_evals[::2]
  poly_evals_o = poly_evals[1::2]
  y_e = IFFT_recursion_part(poly_evals_e, q, inverse_lookup)
  y_o = IFFT_recursion_part(poly_evals_o, q, inverse_lookup)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega**j)*y_o[j]) % q
    y[j + int(n/2)] = int(y_e[j] - (omega**j)*y_o[j]) % q
  return y

print(f'evaluation form of phi(x) is {poly_evals}, after IFFT(use exist omega lookup table), we can get the coefficient form of phi(x) is {IFFT_using_exist_omega(poly_evals, q, inverse_lookup, inverse_omega_dict)}')

evaluation form of phi(x) is [14, 10, 0, 1], after IFFT(use exist omega lookup table), we can get the coefficient form of phi(x) is [2, 3, 5, 4]


## Polynomial Representations, FFT and IFFT(using real numbers for KZG usage)

### Polynomial Representations

Real proving system only consider polynomials over a finite field, and so we will restrict：
- each coefficient $p_i$
- each evaluation point $\Phi(x_i)$ to be elements of a finite field $F_q$
- the evaluation domain(the $\omega_i$'s) to be a mulyiplicative subgroup of $F_q$

For BLS12-381:
- the finite field modulus $q$ is pre-determined, which is 'curve order'
- "$n^{th}$ roots of unity" are confirmed, can be get using the function 'find_nth_root_of_unity($n$, $q$)'

In [9]:
from py_ecc.bls12_381 import curve_order
from py_ecc.fields.field_elements import FQ as Field

# construct a data structure F_q
class FQ(Field):
    field_modulus = curve_order
q = curve_order

# For example, assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 and some points on the polynomials
poly_coefs = [FQ(2), FQ(3), FQ(5), FQ(4)]
poly_evals = [FQ(14), FQ(52435875175126190475982595682112313518914282969839895044333406231173219221502), FQ(0), FQ(3465144826073652318776269530687742778270252468765361963005)]

def find_nth_root_of_unity_realnum(n, q):
  '''
  Args:
  n: nth root of unity, which is the degree of polynomial
  q: finite field q

  Returns:
  omega: the nth root of unity, which is a element in finite field
  '''
  omega = FQ(5) ** ((q - 1) // n)
  return omega

omega = find_nth_root_of_unity_realnum(len(poly_coefs), q)
evaluation_domain = [omega**0, omega**1, omega**2, omega**3]

print(f'coefficient form of phi(x) is {poly_coefs}')
print(f'evaluation form of phi(x) is {poly_evals}, defined over evaluation domain {evaluation_domain}')

coefficient form of phi(x) is [2, 3, 5, 4]
evaluation form of phi(x) is [14, 52435875175126190475982595682112313518914282969839895044333406231173219221502, 0, 3465144826073652318776269530687742778270252468765361963005], defined over evaluation domain [1, 3465144826073652318776269530687742778270252468765361963008, 52435875175126190479447740508185965837690552500527637822603658699938581184512, 52435875175126190475982595682112313518914282969839895044333406231173219221505]


### Fast Fourier Transformation(FFT)

In [10]:
import random

class FQ(Field):
    field_modulus = curve_order

q = curve_order
poly_coefs = [FQ(2), FQ(3), FQ(5), FQ(4)]

def FFT(poly_coefs, q):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_coefs)
  if n == 1:
    return poly_coefs

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = find_nth_root_of_unity_realnum(n, q)
  # print(f'{n}^th of unity is {omega}')
  poly_coefs_e = poly_coefs[::2]
  poly_coefs_o = poly_coefs[1::2]
  y_e = FFT(poly_coefs_e, q)
  y_o = FFT(poly_coefs_o, q)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = y_e[j] + (omega**j)*y_o[j]
    y[j + int(n/2)] = y_e[j] - (omega**j)*y_o[j]
  return y

poly_evals = FFT(poly_coefs, q)
print(f'coefficient form of phi(x) is {poly_coefs}, after FFT, we can get the evaluation form of phi(x) is {poly_evals}')

coefficient form of phi(x) is [2, 3, 5, 4], after FFT, we can get the evaluation form of phi(x) is [14, 52435875175126190475982595682112313518914282969839895044333406231173219221502, 0, 3465144826073652318776269530687742778270252468765361963005]


### Inverse Fast Fourier Transformation(IFFT)

In [11]:
def IFFT(poly_evals, q):
  recursion_result = IFFT_recursion_part(poly_evals, q)
  # all elenent in result mutiply 1/n, which is inverse of n
  IFFT_final_result = [ x * (FQ(1)/FQ(len(poly_evals))) for x in recursion_result]
  return IFFT_final_result

def IFFT_recursion_part(poly_evals, q):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial
  lookup: pre-determined omega list

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_evals)
  if n == 1:
    return poly_evals

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = find_nth_root_of_unity_realnum(n, q)
  # inverse omega
  omega_inv = omega**(n-1)
  poly_evals_e = poly_evals[::2]
  poly_evals_o = poly_evals[1::2]
  y_e = IFFT_recursion_part(poly_evals_e, q)
  y_o = IFFT_recursion_part(poly_evals_o, q)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega_inv**j)*y_o[j])
    y[j + int(n/2)] = int(y_e[j] - (omega_inv**j)*y_o[j])
  return y

print(f'evaluation form of phi(x) is {poly_evals}, after IFFT(use exist omega lookup table), we can get the coefficient form of phi(x) is {IFFT(poly_evals, q)}')

evaluation form of phi(x) is [14, 52435875175126190475982595682112313518914282969839895044333406231173219221502, 0, 3465144826073652318776269530687742778270252468765361963005], after IFFT(use exist omega lookup table), we can get the coefficient form of phi(x) is [2, 3, 5, 4]


# KZG Implementation(Python Version)

The KZG Commitment Scheme is a commitment scheme that allows to commit to a polynomial $\Phi(x) = \phi_0 +\phi_1x+\phi_2x^2+...+\phi_lx^l$, where $\Phi(x) \in F_p[x]$ . 'to commit' means proving that you know the polynomial $\Phi(x)$ without revealing it.

The KZG commitment scheme consists of 4 steps:
1. Setup
2. Commit to Polynomials
3. Prove an Evaluation
4. Verify an Evaluation Proof

## Step 1: Setup

The first step is an one-time trusted setup and once it has done once, the following steps can be done repeatedly
1. Let $G_1$ and $G_2$ be pairing-friendly elliptic curve groups, determined by curve BLS12-381
2. Let $g_1$ be a generator of $G_1$ and $g_2$ be a generator of $G_2$
3. Let $l$ be the maximum degree of the polymonials we want to commit to ($l < p$)
4. Pick a random field element as secret parameter $\tau \in F_p$(usually done by MPC, to simplify, we just randomly choose one here)
5. Compute pp(public parameters, including proving key $pk$, and verifying key $vk$)$$pk = (g_1, g_1^\tau, g_1^{\tau^2},...,g_1^{\tau^l}), vk = g_2^\tau$$ and release it publicly
6. Discard secret parameter $\tau$ once the setup ceremony is done so that nobody can figure out its value

In [12]:
from py_ecc.bls12_381 import G1, G2, Z1, multiply, add

# 1. Let G1,G2 be pairing-friendly elliptic curve groups, which are determined by the curve

# 2. Let g be a generator of G
g1 = G1
g2 = G2

# 3. Let l be the maximum degree of the polymonials, which is 16
l = 16

# 4. Pick a random field element from field F_p and p = curve order as secret parameter t
p = q
# t = 0 will lose all the security, so t cannot be 0
t = FQ(random.randint(1, p-1))
print(f'secret parameter 𝜏 is {t}')

# 5. Compute pp(public parameters)
def compute_public_parameters(g1, g2, t, l):

    pk = []
    accumulated = 1
    t_scalar = t
    for i in range(l + 1):
        # calculate g1, g1^t, g1^{t^2}...
        pk.append(multiply(g1, int(accumulated)))
        # calculate the exponential t, t^2, t^3, ...
        accumulated = accumulated * t_scalar
    vk = multiply(g2, int(t))
    return pk, vk

pk, vk = compute_public_parameters(g1, g2, t, l)
print(f'proving key pk: {pk}')
print(f'verifying key vk: {vk}')

secret parameter 𝜏 is 19236434084501802835586430447143790653798697955050606486856288100650154189927
proving key pk: [(3685416753713387016781088315183077757961620795782546409894578378688607592378376318836054947676345821548104185464507, 1339506544944476473020471379941921221584933875938349620426543736416511423956333506472724655353366534992391756441569), (3245396875867694259546259215218961434809748960240476390443279676478077010380011418759179544715147620250627945204959, 701299001784294637130952544202702502341065405028758036148365841255453237678757561528420141802596686865045393282096), (3841601640763920008428422477895445442753415014207083355804395887624759548685970808817355465055948811688409739562452, 109321603549825995895556058613772404786243195421732920312108581101387347585399362765649987453710034612137920718222), (2423713723240725056725430003883902926975381849368267614375341719973875258392148380910379946286305564976078733481044, 17637714952403041119564480824272199418381499308807455698031

## Step 2: Commit to Polynomials
In reality, we arithmetize circuits and use Plonkish to get polynomials in this step
1. Given a polynomial $\Phi(x) = \sum_{i=0}^l \phi_i x^i$
2. Compute and output commitment $c = g^{\Phi(\tau)}$
   - Wait! $\tau$ has already been discard right? How can committer compute $\tau$?
   - Although he cannot compute $\Phi(\tau)$ directly, he can use public parameters to help with it:
$$\prod_{i=0}^l(g^{\tau^i})^{\phi_i} = g^{\sum_{i=0}^l \phi_i \tau^i} = g^{\Phi(\tau)}$$

In [13]:
# assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 or some points on the polynomials
# we can use FFT or IFFT to do the transformation between the two forms
poly_coefs = [FQ(2), FQ(3), FQ(5), FQ(4)]

In [14]:
# compute commitment of the polynomial
def poly_commitment(pk, g1, poly_coefs):

  com = Z1
  for i in range(len(poly_coefs)):
    com = add(multiply(pk[i], int(poly_coefs[i])), com)
  return com

com = poly_commitment(pk, g1, poly_coefs)
print(f'commitment of polynomial is {com}')

commitment of polynomial is (3967200454875335354292002055990975399352034098957431123280431949250431779646565907329395898388132959704783209660343, 1861546273356388296771394305016094083831235419681889746027899896894573362188734361887345555074770191213090795082140)


## Step 3: Prove an Evaluation
In this period, the Verifier will ask Prover to 'OPEN' the commitment $c$ to a random specific point $a \in F_p$, in other word, Prover have to evaluation $\Phi(x)$ and commit the result in the form of opening triplet $OT = (a, b, \pi)$
1. Given an evaluation $\Phi(a) = b$
2. Compute and output proof of the evaluation $\pi = g^{q(\tau)}$, where $q(x) := \frac{\Phi(x)-b}{x-a}$
    - $q(x)$ is quotient polynomial: if $\Phi(a) = b$, that means $a$ is a root of $\Phi(x) - b$
    - so $\Phi(x) - b$ can be expressed as $\Phi(x) - b = q(x)(x-a)$, $q(x)$ is a polynomial
    - on the other hand, $q(x)$ exists if and only if $\Phi(a) = b$, so the existence of this quotient polynomial therefore serves as a proof of the evaluation

In [15]:
# Verifier first choose the random point a
def evaluation_poly(a, poly_coefs):
  b = FQ(0)
  for i in range(len(poly_coefs)):
    b += (a**i)*(poly_coefs[i])
  return b

# according to defination
def random_generate_a(poly_coefs, p):
  # create a list of omega
  n = len(poly_coefs)
  omega = find_nth_root_of_unity_realnum(n, p)
  omega_list = []
  for i in range(n):
    omega_list.append(omega**i)

  # a cannot in omega list, otherwise x-a will be 0
  a = FQ(random.randint(0, p))
  while a in omega_list:
    a = FQ(random.randint(0, p))

  return a

a = random_generate_a(poly_coefs, p)
b = evaluation_poly(a, poly_coefs)
print(f'random element we chosen is {a}, the evaluation b is {b}')

random element we chosen is 21408353753229090355976215505661384156895171201447048933451914479248483933088, the evaluation b is 13728034202807892556637774915625852320179003930128857614885881403784503374856


In [16]:
# compute q(x)
# 1. if poly in coefficient form, convert poly_coefs to evals
# 2. calculate quotient polynomial in evaluation form
# 3. convert quotient poly evals to coefs if needed

def quotient_poly(a, b, poly_evals, p):
  '''Args:
  a: randomly sampled field element
  b: evaluation on a
  poly_evals: input of poly need to be in evaluation form

  Returns:
  q_poly_evals: q poly in evaluation form
  '''
  q_poly_evals = []
  n = len(poly_evals)
  omega = find_nth_root_of_unity_realnum(n, p)

  for i in range(len(poly_evals)):
    q_poly_evals.append((poly_evals[i]-b)/(omega**i-a))

  return q_poly_evals

q_poly_evals = quotient_poly(a, b, poly_evals, p)
print(f'quotient polynomial in evaluation form is {q_poly_evals}')

quotient polynomial in evaluation form is [50610538997614190428693464310459736899479594840373960732083815445129140889708, 8075751446711095083258985198202264013002115197265266503881414414374286652255, 36651334497160039019226961789726561157389882730380482732279475710957012978533, 26750246872936943885213700393798068206176809872961539137878218041773286031457]


In [17]:
# convert quotient polynomial from evaluation form to coefficient form
q_poly_coefs = IFFT(q_poly_evals, q)
print(f'quotient polynomial in coefficient form is {q_poly_coefs}')

quotient polynomial in coefficient form is [17412999159824019484236342796000166109589462535113402820879816228073786341860, 33197539837790170944457121514459570789890132305260557911203999217055354547844, 4, 0]


In [18]:
# compute proof of the evaluation pi
pi = poly_commitment(pk, g1,  q_poly_coefs)
print(f'proof of the evaluation pi is {pi}')

proof of the evaluation pi is (1356687143464276699291542560776636431355956194199428075655054085673287461850343757435811500819614854308074538014673, 559338417176675560661484463066492561039870137354699661368970219454796754518738514901106020151588573926245689044303)


## Step 4: Verify an Evaluation Proof
1. Given a commitment $c = g^{\Phi(\tau)}$, and an evaluation $\Phi(a) = b$, and a proof $\pi = g^{q(\tau)}$
2. Verify that $e(\frac{c}{g^b}, g) = e(\pi,\frac{g^\tau}{g^a})$, where $e$ is a non-trivial bilinear mapping
    - the purpose of verification: $\Phi(x) - b = q(x)(x-a)$, checking this equality holds for $x = \tau$
    - according to the definition of bilinear mapping, it is equivalent to: $e(g_1, g_2)^{\Phi(\tau) - b} = e(g_1, g_2)^{q(\tau)(\tau-a)}$, that is $e(g_1^{\Phi(\tau) - b}, g_2) = e(g_1^{q(\tau)}, g_2^{\tau-a})$
    - that is $e(com-g_1^b, g_2) = e(\pi, vk - g_2^a)$

In [19]:
from py_ecc.bls12_381 import pairing, neg

# now it is time to do the varification
print(f'result of verification: {pairing(g2, add(com, neg(multiply(g1, int(b))))) == pairing(add(vk, neg(multiply(g2, int(a)))), pi)}')

result of verification: True


# Plonk Implementation(Python Version)

## Problem Definition

We take **Square-Fibonacci** as an example to demonstrate the process of proof generation

Defination of Square-Fibonacci Problem:
- Let $f_0 = 1, f_1 = 1$
- For $i \ge 2$, define $f_i:=(f_{i-2})^2+(f_{i-1})^2 \ mod \ q$
    - $q$ is a large prime integer, used to bound the size of each element, so that it can be represented by some predetermined number of bits.

Let $n$ be some very large integer. For convenience, we assume $n$ is a power of 2

Let $k$ be the $n^{th}$ Square-Fibonacci number

**Our goal**: generate an efficiently-verifiable proof $\pi$ showing that indeed $k$ is the $n^{th}$ Square-Fibonacci number(i.e. $f_n = k$)

## Phases of Proof Generation
The Plonk-based proof generation consists of 3 steps:
1. Filling in the trace table
2. Committing to the trace table
3. Proving the trace table's correctness

In [20]:
# some basic statement
# q here is equialent to the q above
q = curve_order

# assume we hope to prove '8th Square-Fibonacci number is k'
n = 8
k = FQ(317754178345286893212434)

# according to defination, f(0) = 1, f(1) = 1
f_0 = FQ(1)
f_1 = FQ(1)

## Step 1: Filling in the Trace Table
The trace table is a 2-dimensional matrix where 'witness' or 'trace' is written down, that is (n rows * 5 cols)
- 5 columns:
    - $A, B, C$: represent witness data / private input, each row lists 3 sequential Sequare-Fibonacci numbers
        - e.g. the $i^{th}$ row $(f_i, f_{i+1}, f_{i+2})$ is a witness for $(i+2)^{th}$ Sequare-Fibonacci number
    - $S$: represents selector column, indicating a certain mathmatical relation should hold over the element of the row.
        - $1$ represents the first 3 elements of the row $(a, b, c)$ must satisfy $c = a^2 + b^2 \ mod \ q$
        - $0$ represents the condition does not need to be satisfied.
    - $P$: represents public inputs, inputs to the circuit that are public known.
        - e.g. the first two values of the sequence $f_0, f_1$ and $k$ as the value to be proved
- n rows: left a blank row, so that the height of the table becomes $n$, an even power of 2
    - $1^{st}$ row: $f_0, f_1, f_2, 1, f_0$
    - $2^{nd}$ row: $f_1, f_2, f_3, 1, f_1$
    - $3^{rd}$ row: $f_2, f_3, f_4, 1, k$
    - ...
    - $(n-2)^{th}$ row: $f_{n-2}, f_{n-1}, f_n, 1, "" $
    - $(n-1)^{th}$ row: $"", "", "", 0, ""$

Next step is to fill in the trace table: either copy or compute over $F_q$

In [21]:
# generate witness/fill in the trace table
def witness_generation(f_0, f_1, k, n):

    trace_table = []

    # init col A, B, C, S
    f_a = f_0
    f_b = f_1
    f_c = f_b
    S = FQ(1)
    placeholder = FQ(0)

    for i in range(n-1):
        f_a = f_b
        f_b = f_c
        f_c = f_a**2 + f_b**2
        trace_table.append([f_a, f_b, f_c, S, placeholder])

    # add a blank row to get n row
    S = FQ(0)
    trace_table.append([placeholder, placeholder, placeholder, S, placeholder])

    # add public parameters
    trace_table[0][4] = f_0
    trace_table[1][4] = f_1
    trace_table[2][4] = k

    return trace_table

trace_table = witness_generation(f_0, f_1, k, n)

print(f"trace table is: {trace_table}")

trace table is: [[1, 1, 2, 1, 1], [1, 2, 5, 1, 1], [2, 5, 29, 1, 317754178345286893212434], [5, 29, 866, 1, 0], [29, 866, 750797, 1, 0], [866, 750797, 563696885165, 1, 0], [750797, 563696885165, 317754178345286893212434, 1, 0], [0, 0, 0, 0, 0]]


## Step 2: Commit to the Trace Table

### interpret the trace table columns as polynomials

Each column can be considered as a length-$n$ vector of finite field elements $\rightarrow$ this vector can be regarded as the evaluation form of a polynomial $A(x)$ with degree $(n-1)$: the $i^{th}$ element of $A$ corresponds to the evaluation $A(\omega^i)$, where $\omega \in F_q$ is **$n^{th}$ root of unity** and has order $n$

In [22]:
# inverse trace table and get the evaluation form of column polynomials
inverse_trace_table = list(zip(*trace_table))
inverse_trace_table

[(1, 1, 2, 5, 29, 866, 750797, 0),
 (1, 2, 5, 29, 866, 750797, 563696885165, 0),
 (2, 5, 29, 866, 750797, 563696885165, 317754178345286893212434, 0),
 (1, 1, 1, 1, 1, 1, 1, 0),
 (1, 1, 317754178345286893212434, 0, 0, 0, 0, 0)]

### Commit to column polynomials
Now that we have known how to interpret the columns as polynomials, we can commit to each of them using a polynomial commitment scheme.

In [23]:
com_list = []
col_poly_coefs = []
for i in range(len(inverse_trace_table)):
    col_poly_coef = IFFT(inverse_trace_table[i], p)
    col_poly_coefs.append(col_poly_coef)
    com = poly_commitment(pk, g1, col_poly_coef)
    com_list.append(com)
print(com_list)

[(2307159154040568909411228902892145857040518780358010365020284527506998437351829307275257588199113171973139929867167, 2683925305857826639088294353084933537753140135009600843422667209828551698986542607410091916149918452961829501673029), (3293482013260097456805373684702648190845602291441732960247180672651830182640286468738171913772160808832670719747109, 1821470081977977869624959631274008260842315022148971194028324122121385354859420792970576751419148496533777877399366), (1293321581578800103931746816845997978425444554254242862190148407240203557946034445842277247579979467358805569447704, 3475674689198666123514229505921189979489959947390062108350940586267753192625615703542826669277751915738374954553494), (2419900308442263139675115885495422891453439499560076786905960913961251991055770742990133442453602384880793108717894, 3223794919315569795542502174068234973832998554210336612800322216616753286086523802258830124106274480299138772667598), (191545997617149790206161417326702805398075273495143051

## Step 3: Proving the Trace Table's Correctness
### Define the constraints of the trace table
In order to ensure the original trace table to be valid, we should have the following constraints:
- Square-Fibonaci constraints:
    - each selected row $i$'s first three elements is $(a,b,c)$ must satisfy $c_i = a_i^2 + b_i^2 \ mod \ q$
- Wiring constraints:
    - for consecutive rows with value $[a_i, b_i, c_i]$ and $[a_{i+1}, b_{i+1}, c_{i+1}]$, we require $a_{i+1} = b_i$ and $b_{i+1} = c_i$
- Public input constraints:
    - the first row must start with the first two Square-Fibonacci numbers: $a_0 = p_0, b_0 = p_1$
    - the $n^{th}$ Square-Fibonacci must match the claimed result value: $c_{n-2} = p_2$

The above constraints can be represented by one or more relations between the column polynomials. For example, Square-Fibonaci constraints can be expressed as $S(x)·(A(x)^2 + B(x)^2 - C(x))=0, for\ all\ x\in \{w^0, w^1, ..., w^{n-1}\}$. For shorten, we will label left-hand side $\phi_0(x):=S(x)·(A(x)^2 + B(x)^2 - C(x))$  All our constraints can be expressed as $\phi_i(x) = 0,\ for\ all\ x\in \{w^0, w^1, ..., w^{n-1}\}$

In [24]:
def square_fib_constraint_poly(inverse_trace_table, p):

  a_poly_coefs = IFFT(inverse_trace_table[0], p)
  b_poly_coefs = IFFT(inverse_trace_table[1], p)
  c_poly_coefs = IFFT(inverse_trace_table[2], p)
  s_poly_coefs = IFFT(inverse_trace_table[3], p)

  # construct a polynomial phi_0(x) = S(x)(A(x)^2+B(x)^2-C(x))
  phi_0_coefs = mul_poly(s_poly_coefs,sub_poly(add_poly(mul_poly(a_poly_coefs, a_poly_coefs), mul_poly(b_poly_coefs, b_poly_coefs)),c_poly_coefs))

  return phi_0_coefs


def mul_poly(poly1_coefs, poly2_coefs):
  multiply_poly = [0]*(len(poly1_coefs)+len(poly2_coefs)-1)
  for o1,i1 in enumerate(poly1_coefs):
    for o2,i2 in enumerate(poly2_coefs):
      multiply_poly[o1+o2] += i1*i2
  return multiply_poly

def add_poly(poly1_coefs, poly2_coefs):

  if len(poly1_coefs) >= len(poly2_coefs):
    addition_poly = poly1_coefs
    min_length = len(poly2_coefs)
  else:
    addition_poly = poly2_coefs
    min_length = len(poly1_coefs)

  for i in range(min_length):
    addition_poly[i] = poly1_coefs[i] + poly2_coefs[i]

  return addition_poly

def sub_poly(poly1_coefs, poly2_coefs):

  if len(poly1_coefs) >= len(poly2_coefs):
    subtraction_poly = poly1_coefs
    min_length = len(poly2_coefs)
  else:
    subtraction_poly = poly2_coefs
    min_length = len(poly1_coefs)

  for i in range(min_length):
    subtraction_poly[i] = poly1_coefs[i] - poly2_coefs[i]

  return subtraction_poly

square_fib_constraint_poly(inverse_trace_table, p)

[9217243683127650670215423136204564307406542431733373805594493117736508689447,
 6666845327329705532963430625265951311576613171241245242132100072955433688671,
 44267534341411372821110843579994475498172624576994019134899687329660324137538,
 27734915317295573979553823553557321860270268471208809659398702095994161952080,
 37702370013672502632704303140624342804248711303899845502198214997157027878207,
 11583562288821433247490937116774158461617493607772389006562557827980992423716,
 19431544404868491752428433057080910022889750308878986243969634670562912518450,
 12399440026714068747233115926566936096429177141939361413989904821957360748254,
 20041303549030907216090313199445756936739800542200228711060022339667450807871,
 1640433480202112741396001466119286110343076964539974284673711265395784145821,
 13311292830147122275312324513496984175656020478185536494792425391434485506145,
 41059692124944929433452274305014365636083572356756835655701111647528539392349,
 487287176898972862051909273570054856936489

In [25]:
# wiring constraints
# using Fi_1 to represent a_i+1 = b_i
def wiring_constraint_poly(inverse_trace_table, p):

  a_poly_coefs = IFFT(list(inverse_trace_table[0][1:])+[FQ(0)], p)
  b_poly_coefs = IFFT(inverse_trace_table[1], p)
  s_poly_coefs = IFFT(list(inverse_trace_table[3][1:])+[FQ(0)], p)

  # construct a polynomial phi_0(x) = S(x+1)(A(x+1)-B(x))
  phi_1_coefs = mul_poly(s_poly_coefs, sub_poly(a_poly_coefs, b_poly_coefs))

  return phi_1_coefs

wiring_constraint_poly(inverse_trace_table, p)

[11470347694558854167379193236165680026994808359490420773694550340558718051128,
 31658831632200537539783533621391251008881967401427553257134119658570500984004,
 9251157176012360485135106648195872832425943262155855154068869944562980101270,
 4629573589425733109598056930899130194516745852141088748221668437883354628737,
 27092786656927993916917219376235083497636322707752077888330974664656646497822,
 16147469831419210761107806895310410948843277724729960615923746116523503448252,
 47225732227417011674567510410490426723757052077807989737374736443825102691387,
 0,
 40965527480567336312068547272020285810695744141037217048909108359379863133385,
 20777043542925652939664206886794714828808585099100084565469539041368080200509,
 43184717999113829994312633859990093005264609238371782668534788755375601083243,
 47806301585700457369849683577286835643173806648386549074381990262055226555776,
 25343088518198196562530521131950882340054229792775559934272684035281934686691,
 3628840534370697971833993361287555488

In [26]:
# wiring constraints
# using Fi_2 to represent b_i+1 = c_i
def wiring_constraint_poly2(inverse_trace_table, p):

  b_poly_coefs = IFFT(list(inverse_trace_table[1][1:])+[FQ(0)], p)
  c_poly_coefs = IFFT(inverse_trace_table[2], p)
  s_poly_coefs = IFFT(list(inverse_trace_table[3][1:])+[FQ(0)], p)

  # construct a polynomial phi_0(x) = S(x+1)(A(x+1)-B(x))
  phi_2_coefs = mul_poly(s_poly_coefs, sub_poly(b_poly_coefs, c_poly_coefs))

  return phi_2_coefs

wiring_constraint_poly2(inverse_trace_table, p)

[36049664182899255954620321599377851513412254844112750973250561136337128325687,
 26478744040510257060869321766053525138872800379087851350621017645042397135506,
 32410835611437218704466690526562532753646862728670419077503224720809139370953,
 11748757413644117521565380048665272397719182982198598038819999551729010043282,
 5425566726670899114035402883671431159983196801318069536656707248542603833947,
 45997037633194194370717268143927422353966899878804396217539101952889432168702,
 37540168226135780995704033570283361978050109689519990765762645485100182660726,
 0,
 16386210992226934524827418908808114324278297656414886849353097563601452858826,
 25957131134615933418578418742132440698817752121439786471982641054896184049007,
 20025039563688971774981049981623433084043689771857218745100433979129441813560,
 40687117761482072957882360459520693439971369518329039783783659148209571141231,
 47010308448455291365412337624514534677707355699209568285946951451395977350566,
 643883754193199610873047236425854348

In [27]:
constraints_poly = []

# Square-Fibonaci constraints
constraints_poly.append(square_fib_constraint_poly(inverse_trace_table, p))

# wiring constraints
# using Fi_1 to represent a_i+1 = b_i
constraints_poly.append(wiring_constraint_poly(inverse_trace_table, p))

# using Fi_2 to represent b_i+1 = c_i
constraints_poly.append(wiring_constraint_poly2(inverse_trace_table, p))

constraints_poly

[[9217243683127650670215423136204564307406542431733373805594493117736508689447,
  6666845327329705532963430625265951311576613171241245242132100072955433688671,
  44267534341411372821110843579994475498172624576994019134899687329660324137538,
  27734915317295573979553823553557321860270268471208809659398702095994161952080,
  37702370013672502632704303140624342804248711303899845502198214997157027878207,
  11583562288821433247490937116774158461617493607772389006562557827980992423716,
  19431544404868491752428433057080910022889750308878986243969634670562912518450,
  12399440026714068747233115926566936096429177141939361413989904821957360748254,
  20041303549030907216090313199445756936739800542200228711060022339667450807871,
  1640433480202112741396001466119286110343076964539974284673711265395784145821,
  13311292830147122275312324513496984175656020478185536494792425391434485506145,
  41059692124944929433452274305014365636083572356756835655701111647528539392349,
  48728717689897286205190927357

### Combine constraints

- **Naive way to proof the contraints**

    In general, we have $m$ constraint polynomials $\phi_0(x), \phi_1(x), ..., \phi_{m-1}(x)$. Sample a random field element $\gamma \in F_q$, and then take a random linear combination of the individual constraints:$$\phi(x) := \gamma^0·\phi_0(x) + \gamma^1·\phi_1(x)+...+\gamma^{m-1}·\phi_{m-1}(x)$$
    and we need the constraints satisfied at every row, that is $\phi(\omega^i) = 0 \ for \ all\ 0\le i <n$, in this case we need $n$ evaluation proofs


- **Using quotient polynomial**

    we can prove such constraint $\phi(x)$ using only one evaluation proof:
    - quotient polynomial:
        \begin{aligned}
        \phi(\omega^i) = 0 \ for \ all\ 0\le i <n\ &{\Leftrightarrow}\ (x-\omega^i)|\phi(x)\ for \ all\ 0\le i<n\\
        &{\Leftrightarrow}\ \prod^{n-1}_{i=0}(x-\omega^i)|\phi(x)\ for \ all\ 0\le i <n\\
        &{\Leftrightarrow}\ (x^n-1)|\phi(x)\\
        &{\Leftrightarrow}\ \exists Q(x)\ s.t.\phi(x)=Q(x)·(x^n-1)
        \end{aligned}
      now we just need to prove there exists a polynomial $Q(x)$
    - compute the quotient polynomial:
        $$
        Q(x):= {\phi(x) \over {x^n-1}} = {{\gamma^0·\phi_0(x) + \gamma^1·\phi_1(x)+...+\gamma^{m-1}·\phi_{m-1}(x)} \over {x^n-1}}
        $$
      degree of $Q(x)$ is $2n-3$, so we need at least $2n-2$ evaluation points to represent it
      
      we make it a round number and use $2n$ evaluation points, our previous evaluation domain do not work anymore, because the order of $\omega$ is only $n$. We therefore need to pick some other element $\beta \in F_q$ with order $2n$. Then we evaluate $Q(x)$ over the evaluation domain $\{\beta^0, \beta^1,...,\beta^{2n-1}\}$

In [28]:
# define Phi(x)
def Phi(constraints_poly, n, p):
  # randomly gerenate gamma in F_p
  gamma_list = []
  Phi_poly_coefs = [0] * n

  for i in range(len(constraints_poly)):
      gamma = FQ(random.randint(0, p))
      gamma_list.append(gamma)
      phi_i = [gamma * constraints_poly[i][j] for j in range(len(constraints_poly[i]))]
      Phi_poly_coefs = add_poly(phi_i, Phi_poly_coefs)

  return Phi_poly_coefs, gamma_list

Phi_poly_coefs, gamma_list = Phi(constraints_poly, len(constraints_poly[0]), p)
print(f'the combination of contraints is {Phi_poly_coefs}')
print(f'the ramdom generated list of gamma is {gamma_list}')

the combination of contraints is [519995836690240634103880661333640138066485859381822666885119514291780995286, 25496475147381924792196934333513768827802154386201595119487714728805403625492, 28715305596602092172167018078253418977638473454592581647381684112477896097786, 1365921097234892215969681626733228772549702694016562102035965045273949707755, 30696131141730217885907439861841237711785657350851990672479596897582677655400, 2380701770934730876619368416879596749337296441049480865218838261154066161716, 8636564511174624851699491758885756170452475394970429416179896558104050048945, 8132268166320032600996470808689192570661923594654560774038409165509871238318, 19123873602309478329555939980688291773009790407598475790292995522220634056255, 17436820381864156207190849369005198250663182220610321106800448568702102531766, 42363720343758486217033388222411608960548670785472393667193382728081564647147, 13549103083127192581133995134256320095145305094147539192831109912058941300987, 157510923274127363006022

In [29]:
def Quotient_poly(Phi_poly_coefs, n, p):
  '''Args:
  a: randomly sampled field element
  b: evaluation on a
  poly_evals: input of poly need to be in evaluation form

  Returns:
  Q_poly_evals: q poly in evaluation form
  '''
  Q_poly_evals = []

  omega = find_nth_root_of_unity_realnum(2*n, p)

  for i in range(2*n):

    Phi_poly_eval = evaluation_poly(omega**i, Phi_poly_coefs)
    Q_poly_evals.append(Phi_poly_eval/((omega**i)**n-FQ(1)))

  return Q_poly_evals

Q_poly_evals = Quotient_poly(Phi_poly_coefs, n, p)
print(f'quotient polynomial in evaluation form is {Q_poly_evals}')

quotient polynomial in evaluation form is [0, 8231435731486340471051748102571940041365963049365051065948841119779563159473, 0, 4482085480686379382016775985332903504933382189302356843353365864979346231162, 0, 8912788121487567035348612355011332260020864995183029311932181009185731503079, 0, 2052508922924139802603964055274298287239712574442333387322563355192359775501, 0, 14639893252051840520481037387059965669674451136239579915636618729370164420442, 0, 20145060115059820073131643297211289156347061973007030190634954732456747234547, 0, 34219074383043581318865431457291700502332891604038316256277912229641223531942, 0, 7872267636609967553500566697566938924473443238682471528633868437221355409381]


### Committing to the quotient polynomial
Now we have get the evaluation form of $Q(x)$, we can compute its commitment. Note, degree of $Q(x)$ is larger than the column polynomials, so it requires a larger KZG setup

In [30]:
Q_poly_coefs = IFFT(Q_poly_evals, p)
Q_poly_coefs

[35779874388717834404501840244437128805350171454063056806448327111079607620384,
 8718410190932078103595424684502599125331591110305160553400224284351051265883,
 47399797759442338348240564365298787399119611643000015744898520714010072915830,
 32992489129126691530290867821221142966417928797337588507717384305998761242750,
 34093483751269463390024993300621305395239003765861815283387923571951450034406,
 52166116429287350659503427896525711859518866943255666581961875343196403136883,
 21899655331975782813874124374650104833619038552778604203211881070917265567784,
 48369741091966174178949505103841369552359590703200357435584454117183645565354,
 16656000786408356074945900263748837032340381046464581016155331588858973564129,
 43717464984194112375852315823683366712358961390222477269203434415587529918630,
 5036077415683852131207176142887178438570940857527622077705137985928508268683,
 19443386045999498949156872686964822871272623703190049314886274393939819941763,
 18342391423856727089422747207564660442451

In [31]:
Q_com = poly_commitment(pk, g1, Q_poly_coefs)
print(f'commitment of polynomial is {Q_com}')

commitment of polynomial is (871120967902704666220363513284612666757033668801943226106690508948542973435836858607002341484257129858007429502141, 710965719919593128789141539957106105560544514970484654509052741957129781632969188428187535935380055786588303418509)


### Proving an Evaluation
Now we have committed all column polynomials from the trace table and have also committed to the quotient polynomial. Now the prover needs to demonstrate the quotient polynimial really exist and it was computed correctly.

This can be achieve through the following steps:
1. sample a random point $\alpha \in F_q$
2. generate and output KZG proofs for all column polynimials and the quotient polynomials at point $\alpha$ $$ Q(\alpha):= {\phi(\alpha) \over {\alpha^n-1}} = {{\gamma^0·\phi_0(\alpha) + \gamma^1·\phi_1(\alpha)+...+\gamma^{m-1}·\phi_{m-1}(\alpha)} \over {\alpha^n-1}}$$
$\alpha$ is sampled at random, so the property holds at $\alpha$, then it holds everywhere

In [32]:
# Verifier first choose the random point a
a = FQ(random.randint(0, p))
print(f'random point a is {a}')

# calculate evaluation for all polynomials, and proof of the evaluation pi
## for column polynomials
col_evaluation = []
col_q_poly_evals = []
col_pi = []
for poly_coef in col_poly_coefs:
    # compute evaluation at a
    b = evaluation_poly(a, poly_coef)
    col_evaluation.append(b)
    # compute q(x)
    poly_eval = FFT(poly_coef, p)
    q_poly_evals = quotient_poly(a, b, poly_eval, p)
    col_q_poly_evals.append(q_poly_evals)
    # compute proof of the evaluation pi
    q_poly_coef = IFFT(q_poly_evals, p)
    pi = poly_commitment(pk, g1,  q_poly_coef)
    col_pi.append(pi)
print(f'evalutions for column polynomials are {col_evaluation}, and proof of evaluations are {col_pi}')

## for quotient polynomials
# compute evaluation at a
Q_evaluation = evaluation_poly(a, Q_poly_coefs)
# compute q(x)
Q_q_poly_eval = quotient_poly(a, Q_evaluation, Q_poly_evals, p)
# compute proof of the evaluation pi
Q_q_poly_coef = IFFT(Q_q_poly_eval, p)
Q_pi = poly_commitment(pk, g1, Q_q_poly_coef)
print(f'evaluation for quotient polynomial is {Q_evaluation},and proof of evaluations are {Q_pi}')

random point a is 28586676819331591789016301006073149835957332231713141543743323436203368581497
evalutions for column polynomials are [22917849503879836361739413480057959208382713934276593046217568075260123235371, 7867708355382249721560731093429114116836278158266897258224825559346909178035, 1019571242214072688776986709407887304954666658076675088447955654257330445738, 37545219354152755795744734131144318439840020988964859006830047453853889168080, 17854884005351557187736846768234706800651751917034771303413990793771729618336], and proof of evaluations are [(3508032553524116272950125484761740243572239918520456878310347780613776735143384221929695016054049557748139823575415, 1189175241784445229596703282249061522365866510222027218255564077047021615604485604927175195600409017867060271249693), (2887817808143235953337031237578884788005533001033071036155157387831926659823987856255932326546900951062196515617678, 227568375264386833504084706179745041631343177459252980598029810424504459640148666028509

### Verifying the quotient polynomial
the verifier need to check two things:
1. each evaluation proof is correct
2. the quotient polynomial holds at point $\alpha$

In [33]:
# verify each evaluation proof is correct
for i in range(len(com_list)):
    # print(f'verification result of column {i} : {evaluation_verification(com_list[i], col_evaluation[i], a, col_pi[i], g1, g2, vk)}')
    print(f'verification result of column {i} : {pairing(g2, add(com_list[i], neg(multiply(g1, int(col_evaluation[i]))))) == pairing(add(vk, neg(multiply(g2, int(a)))), col_pi[i])}')

# verify the quotient polynomial holds at point a
# assert GT.pairing(Q_com - g1*Scalar(Q_evaluation), g2) == GT.pairing(Q_pi, vk - g2*Scalar(a))
# print(f'verification result:{evaluation_verification(Q_com, Q_evaluation, a, Q_pi, g1, g2, vk)}')
print(f'verification result of quotient polynomial: {pairing(g2, add(Q_com, neg(multiply(g1, int(Q_evaluation))))) == pairing(add(vk, neg(multiply(g2, int(a)))), Q_pi)}')

verification result of column 0 : True
verification result of column 1 : True
verification result of column 2 : True
verification result of column 3 : True
verification result of column 4 : True
verification result of quotient polynomial: True
